In [1]:
import pandas as pd
import numpy as np

#### <데이터를 재배치하는 다양한 기본 연산>

## 1.계층적 색인으로 재형성하기

* stack: 데이터의 컬럼을 로우로 피벗(회전)시킨다.
* unstack: 로우를 컬럼으로 피벗시킨다.

In [2]:
data=pd.DataFrame(np.arange(6).reshape((2,3)),
                 index=pd.Index(['Ohio','Colorado'],name='state'),
                 columns=pd.Index(['one','two','three'],
                                 name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [3]:
# stack메서드 사용하기
result=data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [4]:
# unstack메서드 사용하기
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [5]:
# stack() unstack() 모두 가장 안쪽에 있는 레벨부터 끄집어낸다.
# 레벨숫자나 이름을 전달해서 끄집어낼 단계를 지정할 수 있다.

In [6]:
# level=0지정
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [7]:
# 이름지정
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [8]:
s1=pd.Series([0,1,2,3],index=['a','b','c','d'])
s2=pd.Series([4,5,6],index=['c','d','e'])
data2=pd.concat([s1,s2],keys=['one','two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [9]:
# 모든 값이 하위그룹에 속하지 않을 경우 unstack을 하게 되면 누락된 데이터가 생길 수있다.
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [10]:
# 반면에 stack메서드는 누락된 데이터를 자동으로 걸러낸다.
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [11]:
data2.unstack().stack(dropna=False) # dropna=False를 주어서 누락된 데이터를 포함한다.

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [12]:
df=pd.DataFrame({'left':result,'right':result+5},
               columns=pd.Index(['left','right'],name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [13]:
# DataFrame을 unstack()할때 unstack() 레벨은 결과에서 가장 낮은 단계가 된다.
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [14]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

## 2.긴 형식에서 넓은 형식으로 피벗하기

In [15]:
data=pd.read_csv("macrodata.csv")
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [16]:
# 시간 간격을 나타내보자
periods=pd.PeriodIndex(year=data['year'],quarter=data['quarter'],name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [17]:
columns=pd.Index(['realgdp','infl','unemp'],name='item') #컬럼 만들기
data=data.reindex(columns=columns)
data

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2
...,...,...,...
198,13324.600,-3.16,6.0
199,13141.920,-8.79,6.9
200,12925.410,0.94,8.1
201,12901.504,3.37,9.2


In [18]:
data.index=periods.to_timestamp('D','end')
data

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,13324.600,-3.16,6.0
2008-12-31 23:59:59.999999999,13141.920,-8.79,6.9
2009-03-31 23:59:59.999999999,12925.410,0.94,8.1


In [19]:
ldata=data.stack()
ldata

date                           item   
1959-03-31 23:59:59.999999999  realgdp     2710.349
                               infl           0.000
                               unemp          5.800
1959-06-30 23:59:59.999999999  realgdp     2778.801
                               infl           2.340
                                            ...    
2009-06-30 23:59:59.999999999  infl           3.370
                               unemp          9.200
2009-09-30 23:59:59.999999999  realgdp    12990.341
                               infl           3.560
                               unemp          9.600
Length: 609, dtype: float64

In [20]:
ldata=data.stack().reset_index().rename(columns={0:'value'})
ldata[:10] #긴 형식 데이터

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [21]:
pivoted=ldata.pivot('date','item','value') # 처음 두 인자는 로우와 컬럼 색인으로 사용될 컬럼 이름, 마지막 인자는 df에 채워 넣을 값을 담고 잇는 컬럼이름
pivoted 

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [22]:
ldata['value2']=np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-1.577844
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.022297
2,1959-03-31 23:59:59.999999999,unemp,5.800,0.291841
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-1.002371
4,1959-06-30 23:59:59.999999999,infl,2.340,0.043539
5,1959-06-30 23:59:59.999999999,unemp,5.100,1.626415
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.944471
7,1959-09-30 23:59:59.999999999,infl,2.740,-1.795997
8,1959-09-30 23:59:59.999999999,unemp,5.300,-0.989128
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,0.878359


In [23]:
pivoted=ldata.pivot('date','item')
pivoted[:-5]

value                     value2            \
item                           infl    realgdp unemp      infl   realgdp   
date                                                                       
1959-03-31 23:59:59.999999999  0.00   2710.349   5.8 -0.022297 -1.577844   
1959-06-30 23:59:59.999999999  2.34   2778.801   5.1  0.043539 -1.002371   
1959-09-30 23:59:59.999999999  2.74   2775.488   5.3 -1.795997  0.944471   
1959-12-31 23:59:59.999999999  0.27   2785.204   5.6  0.368248  0.878359   
1960-03-31 23:59:59.999999999  2.31   2847.699   5.2 -1.019794  0.452854   
...                             ...        ...   ...       ...       ...   
2007-06-30 23:59:59.999999999  2.75  13203.977   4.5 -0.307219 -0.021794   
2007-09-30 23:59:59.999999999  3.45  13321.109   4.7 -1.920690 -0.421185   
2007-12-31 23:59:59.999999999  6.38  13391.249   4.8 -0.117280 -0.667741   
2008-03-31 23:59:59.999999999  2.82  13366.865   4.9 -0.213984 -3.322715   
2008-06-30 23:59:59.999999999  8.53  13415.266   5.4  2.339268  0.839431   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  0.291841  
1959-06-30 23:59:59.999999999  1.626415  
1959-09-30 23:59:59.999999999 -0.989128  
1959-12-31 23:59:59.999999999  1.020923  
1960-03-31 23:59:59.999999999 -1.841362  
...                                 ...  
2007-06-30 23:59:59.999999999 -1.647537  
2007-09-30 23:59:59.999999999  1.546489  
2007-12-31 23:59:59.999999999 -0.612889  
2008-03-31 23:59:59.999999999  1.322089  
2008-06-30 23:59:59.999999999 -0.496799  

[198 rows x 6 columns]

In [24]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [25]:
unstacked=ldata.set_index(['date','item']).unstack('item')
unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.022297 -1.577844   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  0.043539 -1.002371   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -1.795997  0.944471   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.368248  0.878359   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -1.019794  0.452854   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2  0.444776  0.722018   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6 -0.463644  1.111768   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  0.291841  
1959-06-30 23:59:59.999999999  1.626415  
1959-09-30 23:59:59.999999999 -0.989128  
1959-12-31 23:59:59.999999999  1.020923  
1960-03-31 23:59:59.999999999 -1.841362  
1960-06-30 23:59:59.999999999  0.008663  
1960-09-30 23:59:59.999999999  1.223603

## 3.넓은 형식에서 긴 형식으로 피벗하기
* pandas.melt
* 하나의 컬럼을 여러 개의 새로운 df로 생성하기 보다는 여러 컬럼을 하나로 병합하고 DataFrame을 입력보다 긴 형태로 만들어낸다.

In [26]:
df=pd.DataFrame({'key':['foo','bar','baz'],
                'A':[1,2,3],
                'B':[4,5,6],
                'C':[7,8,9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [27]:
#'key' 컬럼을 그룹 구분자로 사용할 수 있고 다른 컬럼을 데이터값으로 사용할 수 있다.
# pd.melt를 사용할 때는 반드시 어떤 컬럼을 그룹 구분자로 사용할 것인지 지정해야 한다.

In [28]:
# 'key'를 그룹 구분자로 지정
melted=pd.melt(df,['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [29]:
# 원래 모양으로 되돌릴 수 있다.
reshaped=melted.pivot('key','variable','value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [30]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [31]:
# 데이터값으로 사용할 컬럼들의 집합을 지정할 수도 있다.
pd.melt(df,id_vars=['key'],value_vars=['A','B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [32]:
# 그룹 구분자 없이도 사용가능
pd.melt(df,value_vars=['A','B','C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [33]:
# 그룹 구분자 없이도 사용가능
pd.melt(df,value_vars=['key','A','B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
